In [13]:
import requests
import json
import pandas as pd
from random import randint
import time
import timeit
import glob

datestr = time.strftime("%Y%m%d")

In [103]:
# create function to return .json results from carmax hidden API
def cargurus(pagenum,zipcode):
    options = dict(
        pagenum = pagenum,
        zipcode = zipcode
        )
    
# https://api.carmax.com/v1/api/vehicles?Distance=50&PerPage=20&SortKey=0&StartIndex=40&ExposedDimensions=249+250+1001+1000+265+999+772&ExposedCategories=249+250+1001+1000+265+999+772&Refinements=&Page=3&Zip=40003&platform=carmax.com&apikey=adfb3ba2-b212-411e-89e1-35adab91b600
    
    json_url = 'https://www.cargurus.com/Cars/inventorylisting/ajaxFetchSubsetInventoryListing.action?sourceContext=forSaleTab_false_0/zip=40003&address=Bagdad%2C+KY&latitude=38.2781982421875&longitude=-85.05079650878906&distance=10&selectedEntity=&entitySelectingHelper.selectedEntity2=&minPrice=&maxPrice=&minMileage=&maxMileage=&transmission=ANY&bodyTypeGroup=&serviceProvider=&page=1&filterBySourcesString=&filterFeaturedBySourcesString=&displayFeaturedListings=true&searchSeoPageType=&inventorySearchWidgetType=AUTO&allYearsForTrimName=false&daysOnMarketMin=&daysOnMarketMax=&vehicleDamageCategoriesRaw=&minCo2Emission=&maxCo2Emission=&vatOnly=false&minEngineDisplacement=&maxEngineDisplacement=&minMpg=&maxMpg=&isRecentSearchView=false'
      
    raw = requests.get(json_url).text
    data = json.loads(raw)
    
    return data

In [107]:
len(cargurus(pagenum=1,zipcode=40003)['listings'])

173

In [22]:
output.keys()

dict_keys(['listings', 'styleSet', 'serviceProviders', 'page', 'sellers', 'remainingResults'])

In [3]:
listings = carmax(1)['ResultCount'] # return results for first page
print("# Listings:",listings)

pages = listings//50+2
print("# Pages:",pages)

# Listings: 66929
# Pages: 1340


In [4]:
# create cars dataframe to store scraped results
cars = pd.DataFrame(carmax(1)['Results'])

t0 = timeit.default_timer() # start timer
for i in range(1,pages):
    t1 = timeit.default_timer()
    
    df = pd.DataFrame(carmax(i)['Results'])
    df.to_json('data/'+datestr+'_'+str(i).zfill(4)+'.json')   

#     time.sleep(randint(1,10)/10) # add random delay
    
    print("Page",i,"scraped:",round(timeit.default_timer()-t1,2),"sec. elapsed;",round(i/pages*100,2),"% complete...")  
print("Scrape complete;",round(timeit.default_timer() - t0,2),"sec. elapsed.")

Page 1 scraped: 0.16 sec. elapsed; 0.07 % complete...
Page 2 scraped: 0.6 sec. elapsed; 0.15 % complete...
Page 3 scraped: 0.53 sec. elapsed; 0.22 % complete...
Page 4 scraped: 0.5 sec. elapsed; 0.3 % complete...
Page 5 scraped: 0.52 sec. elapsed; 0.37 % complete...
Page 6 scraped: 0.81 sec. elapsed; 0.45 % complete...
Page 7 scraped: 0.63 sec. elapsed; 0.52 % complete...
Page 8 scraped: 0.59 sec. elapsed; 0.6 % complete...
Page 9 scraped: 0.61 sec. elapsed; 0.67 % complete...
Page 10 scraped: 0.81 sec. elapsed; 0.75 % complete...
Page 11 scraped: 0.2 sec. elapsed; 0.82 % complete...
Page 12 scraped: 0.15 sec. elapsed; 0.9 % complete...


KeyboardInterrupt: 

In [5]:
path = !pwd

In [9]:
# combine results and summarize
cars = pd.DataFrame()

for filename in glob.iglob(path[0]+'/data/*.json'):
    cars = pd.concat([cars,pd.read_json(filename)])
    
pd.to_pickle('/data/'+datestr+'carmax.pkl')

TypeError: to_pickle() missing 1 required positional argument: 'path'

In [7]:
cars.head()

AverageRating  Cylinders                  Description DriveTrain  \
0           4.750          4    2015 Volvo S60 T5 Premier        2WD   
1           5.000          6  2015 Volvo XC60 T6 Platinum        4WD   
10          5.000          4               2015 BMW 320 I        2WD   
11          4.548          4               2013 BMW 328 I        2WD   
12          4.463          6               2011 BMW 328 I        2WD   

   EngineSize ExteriorColor  \
0        2.0L          Gray   
1        3.0L          Gray   
10       2.0L         Black   
11       2.0L           Red   
12       3.0L        Silver   

                                           Highlights InteriorColor  IsNew  \
0   Turbo Charged Engine, Leather Seats, Parking S...         Black  False   
1   Technology Package, Power Liftgate/DeckLid, Au...         Black  False   
10  Run Flat Tires, Leatherette Seats, Parking Sen...         Black  False   
11  Technology Package, Cold Weather Package, Run ...           Tan  False   
12  Premium Package, Cold Weather Package, Run Fla...          Gray  False   

    IsNewArrival  ...   StoreCityName  StoreId      StoreName  \
0          False  ...   Mechanicsburg     6013  Mechanicsburg   
1          False  ...   Mechanicsburg     6013  Mechanicsburg   
10         False  ...   Ellicott City     7663  Ellicott City   
11         False  ...   Ellicott City     7663  Ellicott City   
12         False  ...   Ellicott City     7663  Ellicott City   

    StoreStateAbbreviation TransferFee      TransferText TransferType  \
0                       PA         0.0     Free Transfer     Standard   
1                       PA         0.0     Free Transfer     Standard   
10                      MD        99.0  $99 Transfer Fee     Standard   
11                      MD        99.0  $99 Transfer Fee     Standard   
12                      MD        99.0  $99 Transfer Fee     Standard   

      Transmission                Vin  Year  
0        Automatic  YV126MFK7F1344252  2015  
1        Automatic  YV4902RM2F2689879  2015  
10       Automatic  WBA3B1G51FNT64435  2015  
11  Manual 6 Speed  WBA3A5C50DJ584508  2013  
12       Automatic  WBAPH5G57BNM78960  2011  

[5 rows x 38 columns]